In [2]:
from langchain_core.documents import Document
from langchain_community.document_loaders import JSONLoader

d:\LIFE\_BOOTCAMP_MANDIRI\B9_RAG\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
from langchain_community.document_loaders import JSONLoader

# --- TANTANGAN 1: Lengkapi Fungsi Metadata ---
def ambil_metadata(record, metadata):
    # record adalah satu kotak data pembeli (misal: farhan_99)
    metadata["user"] = record.get("username")   # <--- Isi nama field JSON-nya
    metadata["rating"] = record.get("bintang") # <--- Isi nama field JSON-nya
    return metadata

# --- TANTANGAN 2: Setup Loader ---
loader = JSONLoader(
    file_path='data/data.json',
    
    # Petunjuk: Masuk ke 'data_toko', lalu ke 'komentar_pembeli', lalu pecah list-nya.
    jq_schema='.data_toko.komentar_pembeli[]', # <--- Lengkapi path-nya
    
    # Petunjuk: Field mana yang mau dibaca sebagai teks utama?
    content_key='isi_review', # <--- Isi nama field JSON-nya
    
    metadata_func=ambil_metadata
)

# Jalankan
docs = loader.load()

# Cek Hasil
print(f"Content: {docs[0].page_content}")
print(f"Metadata: {docs[0].metadata}")

Content: Laptopnya kencang buat coding, mantap!
Metadata: {'source': 'D:\\LIFE\\_BOOTCAMP_MANDIRI\\B9_RAG\\json_parsing\\data\\data.json', 'seq_num': 1, 'user': 'farhan_99', 'rating': 5}


In [6]:
from langchain_community.document_loaders import JSONLoader

# --- TANTANGAN 1: Metadata Bersarang ---
def get_forum_meta(record, metadata):
    # record adalah satu kotak diskusi
    
    # Ambil Topik sebagai metadata
    metadata["judul_topik"] = record.get("topic")
    
    # TANTANGAN UTAMA: Ambil username (ada di dalam author)
    # Hint: record.get("author") akan menghasilkan {username:..., level:...}
    # Jadi caranya: record.get("author").get("...")
    metadata["penulis"] = record.get("author").get("username")
    
    return metadata

# --- TANTANGAN 2: Setup Loader ---
loader = JSONLoader(
    file_path='data/forum_data.json',
    
    # Hint: Masuk ke 'data', lalu ke 'discussions', lalu pecah array-nya []
    jq_schema='.data.discussions[]', 
    
    # Hint: Kita mau AI baca curhatan user (content), bukan judulnya.
    content_key='content', 
    
    metadata_func=get_forum_meta
)

# Load data
docs = loader.load()

# --- VERIFIKASI (Tampilkan Semua Data) ---
print(f"Total Diskusi: {len(docs)}")
print("="*40)

for doc in docs:
    print(f"Topik    : {doc.metadata['judul_topik']}")
    print(f"Penulis  : {doc.metadata['penulis']}")
    print(f"Masalah  : {doc.page_content}") # Ini harusnya isi curhatan
    print("-" * 20)

Total Diskusi: 3
Topik    : Error saat install Python
Penulis  : koding_mula
Masalah  : Kenapa muncul error 'Path not found' saat saya ketik python di CMD?
--------------------
Topik    : LangChain susah dimengerti
Penulis  : farhan_dev
Masalah  : Ada yang punya tutorial bahasa Indonesia soal JSONLoader? Saya bingung.
--------------------
Topik    : Laptop panas
Penulis  : gamer_tobat
Masalah  : Laptop saya bunyi kipasnya kencang saat training model YOLO.
--------------------


In [7]:
import numpy as np
from sentence_transformers import SentenceTransformer, util

# 1. Kita load model yang support Bahasa Indonesia
# 'paraphrase-multilingual-MiniLM-L12-v2' adalah model yang populer dan ringan
print("Sedang load model...")
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

# 2. Data Input
kalimat_1 = "Saya suka belajar koding Python"
kalimat_2 = "Aku gemar mempelajari bahasa pemrograman Python"  # Mirip makna
kalimat_3 = "Ibu sedang memasak sayur lodeh di dapur"         # Beda topik

# 3. Proses Embedding (Ubah Teks jadi Angka)
emb_1 = model.encode(kalimat_1)
emb_2 = model.encode(kalimat_2)
emb_3 = model.encode(kalimat_3)

# ---------------------------------------------------------
# MARI KITA LIHAT OUTPUTNYA
# ---------------------------------------------------------

print(f"\n--- CONTOH OUTPUT VECTOR (EMBEDDING) ---")
print(f"Kalimat Asli: '{kalimat_1}'")
print(f"Dimensi Vector: {emb_1.shape} (Artinya ada {len(emb_1)} angka)")
print(f"Tipe Data: {type(emb_1)}")
print("\nIsi Vector (5 angka pertama):")
print(emb_1[:5]) 
print("...\n(dan seterusnya sampai 384 angka)\n")

# 4. Menghitung Kemiripan (Cosine Similarity)
score_mirip = util.cos_sim(emb_1, emb_2)
score_beda  = util.cos_sim(emb_1, emb_3)

print(f"--- HASIL PENCARIAN KEMIRIPAN ---")
print(f"1 vs 2 (Coding vs Coding): {score_mirip.item():.4f}")
print(f"1 vs 3 (Coding vs Masak) : {score_beda.item():.4f}")

Sedang load model...


d:\LIFE\_BOOTCAMP_MANDIRI\B9_RAG\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Farhan Kamil\.cache\huggingface\hub\models--sentence-transformers--paraphrase-multilingual-MiniLM-L12-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package 


--- CONTOH OUTPUT VECTOR (EMBEDDING) ---
Kalimat Asli: 'Saya suka belajar koding Python'
Dimensi Vector: (384,) (Artinya ada 384 angka)
Tipe Data: <class 'numpy.ndarray'>

Isi Vector (5 angka pertama):
[ 0.09522366 -0.21008953 -0.46234402 -0.009131   -0.42144918]
...
(dan seterusnya sampai 384 angka)

--- HASIL PENCARIAN KEMIRIPAN ---
1 vs 2 (Coding vs Coding): 0.9218
1 vs 3 (Coding vs Masak) : 0.0571


In [8]:
import numpy as np
from sentence_transformers import SentenceTransformer, util

# --- STEP 1: LOAD MODEL ---
# Kita pakai model multilingual biar ngerti Indo
print("Sedang memuat model...")
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

# --- STEP 2: KNOWLEDGE BASE (Pura-pura jadi Database) ---
# Anggap ini isi dokumen PDF kampus yang sudah dipecah per kalimat
database_dokumen = [
    "Jadwal kuliah semester ganjil dimulai bulan September.",     # Doc 0
    "Syarat beasiswa prestasi adalah IPK minimal 3.50.",         # Doc 1
    "Cara reset password e-learning: Klik lupa sandi di menu login.", # Doc 2
    "Kantin buka setiap hari jam 8 pagi sampai 5 sore.",         # Doc 3
    "Dilarang merokok di area lingkungan kampus.",               # Doc 4
]

# --- STEP 3: INDEXING (Ubah Database jadi Vector) ---
# Ini biasanya dilakukan sekali di awal (Pre-computation)
print("Sedang mengubah database jadi angka (embedding)...")
doc_embeddings = model.encode(database_dokumen, convert_to_tensor=True)

# --- STEP 4: RETRIEVAL (Proses Pencarian) ---
def cari_jawaban(pertanyaan_user):
    print(f"\nUser Bertanya: '{pertanyaan_user}'")
    
    # 1. Ubah pertanyaan user jadi vector
    query_vec = model.encode(pertanyaan_user, convert_to_tensor=True)
    
    # 2. Hitung kemiripan vector pertanyaan vs SEMUA vector database
    # Ini menghitung cos_sim antara 1 query vs 5 dokumen sekaligus
    skor_kemiripan = util.cos_sim(query_vec, doc_embeddings)[0]
    
    # 3. Cari index dengan skor tertinggi (Argmax)
    # Ini mencari posisi juara 1 (paling mirip)
    best_idx = np.argmax(skor_kemiripan.cpu().numpy())
    best_score = skor_kemiripan[best_idx]
    
    # 4. Tampilkan Hasil
    print(f"--- HASIL PENCARIAN ---")
    print(f"Dokumen Paling Relevan (Index {best_idx}):")
    print(f">> \"{database_dokumen[best_idx]}\"")
    print(f"Skor Confidence: {best_score:.4f}")

# --- STEP 5: TEST CASE ---
# Kasus A: Kata kunci beda tapi makna sama (Sandi vs Password)
cari_jawaban("Gimana kalau aku lupa kata sandi akun belajar?")

# Kasus B: Topik lain
cari_jawaban("Kapan kita mulai masuk kuliah?")

Sedang memuat model...
Sedang mengubah database jadi angka (embedding)...

User Bertanya: 'Gimana kalau aku lupa kata sandi akun belajar?'
--- HASIL PENCARIAN ---
Dokumen Paling Relevan (Index 2):
>> "Cara reset password e-learning: Klik lupa sandi di menu login."
Skor Confidence: 0.5702

User Bertanya: 'Kapan kita mulai masuk kuliah?'
--- HASIL PENCARIAN ---
Dokumen Paling Relevan (Index 0):
>> "Jadwal kuliah semester ganjil dimulai bulan September."
Skor Confidence: 0.5906


In [18]:
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')


database_dokumen = [
    "Farhan adalah mahasiswa semester 3",
    "Farhan adalah mahasiswa Itenad prodi informatika",
    "Farhan memiliki skill di bidang AI"
]

doc_embeddings = model.encode(database_dokumen, convert_to_tensor=True)

def cari_jawaban(pertanyaan_user):
    query_vec = model.encode(pertanyaan_user, convert_to_tensor=True)
    
    
    skor_kemiripan = util.cos_sim(query_vec, doc_embeddings)[0]
    
    best_idx = np.argmax(skor_kemiripan.cpu().numpy())
    best_score = skor_kemiripan[best_idx]
    
    # 4. Tampilkan Hasil
    print(f"--- HASIL PENCARIAN ---")
    print(f"Dokumen Paling Relevan (Index {best_idx}):")
    print(f">> \"{database_dokumen[best_idx]}\"")
    print(f"Skor Confidence: {best_score:.4f}")
    
    
    
pertanyaan = input("Masukkan Pertanyaan tentang Farhan")

cari_jawaban(pertanyaan)

--- HASIL PENCARIAN ---
Dokumen Paling Relevan (Index 2):
>> "Farhan memiliki skill di bidang AI"
Skor Confidence: 0.6256


In [19]:
import chromadb

# 1. Bikin Client (Ibarat buka aplikasi Database-nya)
# Data akan disimpan di folder "my_vectordb" di laptopmu (PERMANEN!)
client = chromadb.PersistentClient(path="./my_vectordb")

# 2. Bikin Koleksi (Ibarat bikin Table di SQL)
collection = client.get_or_create_collection(name="dokumen_kampus")

# 3. Masukkan Data (Ingestion)
# Perhatikan: Kita TIDAK PERLU panggil model embedding manual.
# ChromaDB punya default embedding model di dalamnya (all-MiniLM-L6-v2).
collection.add(
    documents=[
        "Jadwal kuliah semester ganjil dimulai bulan September.",
        "Syarat beasiswa prestasi adalah IPK minimal 3.50.",
        "Cara reset password e-learning: Klik lupa sandi di menu login."
    ],
    ids=["doc1", "doc2", "doc3"] # ID unik tiap dokumen
)

# 4. Pencarian (Retrieval)
# Lihat betapa simpelnya, gak perlu rumus matematika lagi!
results = collection.query(
    query_texts=["Lupa kata sandi akun belajar"],
    n_results=1 # Minta 1 hasil teratas
)

print(results['documents'])

C:\Users\Farhan Kamil\.cache\chroma\onnx_models\all-MiniLM-L6-v2\onnx.tar.gz: 100%|██████████| 79.3M/79.3M [04:35<00:00, 302kiB/s]   


[['Cara reset password e-learning: Klik lupa sandi di menu login.']]


In [21]:
collection.peek()

{'ids': ['doc1', 'doc2', 'doc3'],
 'embeddings': array([[-0.03431543,  0.05837245, -0.00937446, ...,  0.06811058,
         -0.07416835,  0.0167924 ],
        [-0.03097427,  0.11782466, -0.03259042, ...,  0.04557349,
         -0.02315447,  0.02813625],
        [-0.06134126,  0.03819292, -0.0429939 , ...,  0.06528832,
         -0.07310627, -0.01126526]], shape=(3, 384)),
 'documents': ['Jadwal kuliah semester ganjil dimulai bulan September.',
  'Syarat beasiswa prestasi adalah IPK minimal 3.50.',
  'Cara reset password e-learning: Klik lupa sandi di menu login.'],
 'uris': None,
 'included': ['metadatas', 'documents', 'embeddings'],
 'data': None,
 'metadatas': [None, None, None]}

In [22]:
collection.add(
    documents= ['Farhan adalah mahasiswa semester 3'],
    ids = ['idFarhan']
)

In [23]:
results = collection.query(
    query_texts=["Farhan semester berapa"],
    n_results=1 
)

print(results['documents'])

[['Farhan adalah mahasiswa semester 3']]


In [25]:
results = collection.query(
    query_texts=["siapa sih yang semester 3 itu?"]
)

print(results['documents'])

[['Jadwal kuliah semester ganjil dimulai bulan September.', 'Farhan adalah mahasiswa semester 3', 'Syarat beasiswa prestasi adalah IPK minimal 3.50.', 'Cara reset password e-learning: Klik lupa sandi di menu login.']]
